In [12]:
import psycopg2
import urllib
import json
import velovUtils
import pandas as pd
from datetime import timedelta
import datetime
import seaborn as sns
from scipy.stats import exponweib
url = 'https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.json'
conn = psycopg2.connect(database = "velov", 
                        user = "dev", 
                        password = "velov", 
                        host = "localhost" ,
                        port = "5432")

In [21]:
# Create sql statement for retrieve data from database for given station and given hour
station = 11
hour = 9
sqlSelect = "select distinct last_update, available_bike_stands, last_update_fme from stations_data where station_id =" + str(
        station) + " and extract(HOUR FROM last_update) =" + str(hour) + " order by last_update"

In [22]:
# Retrieve all data for station at given hour into a pandas.dataFrame
cur = conn.cursor()
cur.execute(sqlSelect)
data = cur.fetchall()
data
dataAsDF = pd.DataFrame(data)
labels = ['last_update', 'avail_bike_stands', "last_update_fme"]
for columns_label in range(0, len(labels)):
    dataAsDF = dataAsDF.rename(columns={columns_label: labels[columns_label]})

In [23]:
data

[]

In [18]:
dataAsDF

,last_update,avail_bike_stands,last_update_fme
0,2022-05-20 07:32:21,11,2022-05-20 09:41:05
1,2022-05-20 07:42:27,11,2022-05-20 09:44:18
2,2022-05-20 07:42:27,11,2022-05-20 09:45:02
3,2022-05-20 07:45:56,24,2022-05-20 09:48:00
4,2022-05-20 07:45:56,24,2022-05-20 09:50:18
5,2022-05-20 07:45:56,24,2022-05-20 09:53:06
6,2022-05-20 07:49:36,10,2022-05-20 09:54:14
7,2022-05-20 07:55:58,24,2022-05-20 09:57:16
8,2022-05-20 07:55:58,24,2022-05-20 09:59:03


In [19]:
# First add to dataAsDF column for duration between two update
duration = [timedelta(seconds=0)] + list(dataAsDF.last_update[i + 1] - dataAsDF.last_update[i] 
                                         for i in range(0, len(dataAsDF) - 1))
dataAsDF = pd.concat([dataAsDF, pd.DataFrame(duration)], axis=1)
dataAsDF = dataAsDF.rename(columns={0: 'duration_since_last_update'})

In [20]:
dataAsDF

,last_update,avail_bike_stands,last_update_fme,duration_since_last_update
0,2022-05-20 07:32:21,11,2022-05-20 09:41:05,0 days 00:00:00
1,2022-05-20 07:42:27,11,2022-05-20 09:44:18,0 days 00:10:06
2,2022-05-20 07:42:27,11,2022-05-20 09:45:02,0 days 00:00:00
3,2022-05-20 07:45:56,24,2022-05-20 09:48:00,0 days 00:03:29
4,2022-05-20 07:45:56,24,2022-05-20 09:50:18,0 days 00:00:00
5,2022-05-20 07:45:56,24,2022-05-20 09:53:06,0 days 00:00:00
6,2022-05-20 07:49:36,10,2022-05-20 09:54:14,0 days 00:03:40
7,2022-05-20 07:55:58,24,2022-05-20 09:57:16,0 days 00:06:22
8,2022-05-20 07:55:58,24,2022-05-20 09:59:03,0 days 00:00:00


In [ ]:
dataAsDF['real_update'] = dataAsDF['duration_since_last_update'].apply(lambda update: 
                                                                       update <= timedelta(minutes=10))

In [ ]:
dataAsDF.to_csv('test.csv')

In [ ]:
sns.displot(data=dataAsDF.loc[dataAsDF['real_update']==True]['avail_bike_stands'],
            kde=True)

In [ ]:
weibulltheo = exponweib.rvs(1, 0.5, loc=0, scale=1, size=200)
sns.displot(data=weibulltheo,
           kde=True)